## 03. 폼 제어

<br>

### `useRef` 훅 활용
- `useRef` 훅은 주로 비제어 컴포넌트에서 입력값을 DOM에서 직접 읽어올 때 사용 이외에도, **JSX 요소 (또는 DOM 노드)를 참조해 포커스를 주거나 스크롤을 이용하는 동작 등을 제어 컴포넌트 방식에서도 수행 가능**

<br>

- `useState` 훅으로 이메일/비밀번호 값을 실시간으로 관리하는 제어 컴포넌트 방식의 로그인 폼
- `onSubmit` 이벤트는 **기본적으로 폼 데이터를 서버로 전송하려는 동작을 포함 $\rightarrow$ `event.preventDefault()`를 호출해 기본 동작 (페이지 새로고침 또는 서버 전송)을 막아야 함**
  
  $\rightarrow$ **생략시 로그인 버튼을 누르는 즉시 폼이 전송되어 페이지가 새로고침 됨**
- **`ref.current`는 `document.getElementByID()`와 유사한 역할**
  - DOM 요소를 직접 가리키며 `focus()`등 의 메서드를 호출해 동적으로 동작을 제어 가능


<br>

```ts
import { useRef, useState } from "react";

export default function LoginForm() {
  const idRef = useRef<HTMLInputElement>(null);
  const pwRef = useRef<HTMLInputElement>(null);

  const [email, setEmail] = useState("");
  const [password, setPassword] = useState("");

  const changeEmail = (e: React.ChangeEvent<HTMLInputElement>) => {
    setEmail(e.target.value);
  };
  const changePassword = (e: React.ChangeEvent<HTMLInputElement>) => {
    setPassword(e.target.value);
  };

  const submitHandler = (e: React.FormEvent<HTMLFormElement>) => {
    e.preventDefault(); // 기본 동작 막기
    if (email.trim() === "") {
      // email 값이 공백이거나 빈 문자열이면
      alert("이메일을 입력해주세요."); // 경고창을 띄우거나
      idRef.current?.focus(); // 포커스 이동
      return; // 함수 실행 종료
    }
    if (password.trim() === "") {
      // password 값이 공백이거나 빈 문자열이면
      alert("비밀번호를 입력해주세요."); // 경고창을 띄우거나
      pwRef.current?.focus(); // 포커스 이동
      return; // 함수 실행 종료
    }

    alert(`이메일: ${email}, 비밀번호: ${password}`);
    setEmail(""); // 이메일 입력 초기화
    setPassword(""); // 비밀번호 입력 초기화
  };

  return (
    <form onSubmit={submitHandler}>
      <label htmlFor="uid">
        <input
          ref={idRef}
          type="text"
          id="uid"
          placeholder="이메일을 입력하세요."
          value={email}
          onChange={changeEmail}
        />
      </label>
      <label htmlFor="upw">
        <input
          ref={pwRef}
          type="password"
          id="upw"
          placeholder="비밀번호를 입력하세요."
          value={password}
          onChange={changePassword}
        />
      </label>
      <button type="submit">로그인</button>
    </form>
  );
}
```

<br>

### 커스텀 훅
- 여러 입력값을 관리할 때 `useState`과 `setValue` 코드가 반복
  
  $\rightarrow$ 매번 `useState`와 `onChange` 이벤트 핸들러를 작성하는 대신, 하나의 커스텀 훅으로 만들어 간단하게 사용 가능

<br>

- `src/hooks/useInput.ts`
    - **`value` 상태와, `onChange()`함수를 객체로 반환**


```ts
import { useState } from "react";

function useInput(initialValue = "") {
  const [value, setValue] = useState(initialValue);
  const onChange = (event: React.ChangeEvent<HTMLInputElement>) => {
    setValue(event.target.value);
  };
  return {
    value,
    onChange,
  };
}

export default useInput;
```

<br>

- `LoginForm3.tsx`

```ts
import useInput from "../hooks/useInput";

export default function LoginForm3() {
  const { value: email, onChange: changeEmail } = useInput("");
  const { value: password, onChange: changePassword } = useInput("");

  const submitHandler = (e: React.FormEvent<HTMLFormElement>) => {
    e.preventDefault();
    console.log(email, password);
  };

  return (
    <form onSubmit={submitHandler}>
      <label htmlFor="uid">
        <input
          type="text"
          id="uid"
          placeholder="이메일을 입력하세요."
          value={email}
          onChange={changeEmail}
        />
      </label>
      <label htmlFor="upw">
        <input
          type="password"
          id="upw"
          placeholder="비밀번호를 입력하세요."
          value={password}
          onChange={changePassword}
        />
      </label>
      <button type="submit">로그인</button>
    </form>
  );
}
```

<br>

### 커스텀 훅 2
- 체크박스, 라디오 버튼은 사용자의 선택 여부에 따라 `true`혹은 `false`값을 사용
  
  $\rightarrow$ **`value` 대신 `checked` 속성을 제어**

<br>

- `src/hooks/useInputEx.ts`


```ts
import { useState } from "react";

type InputType = "text" | "checkbox" | "radio"; 

// useInput 훅에 전달한 props의 타입 정의
interface UseInputProps<T> {
  initialValue: T; // 초깃값
  validateFn: (value: T) => string | undefined; // 유효성 검사 함수
  type?: InputType; // 입력 필드 타입(기본값 'text')
}

export default function useInputEx<T>({
  initialValue,
  validateFn,
  type = "text",
}: UseInputProps<T>) {

  const [value, setValue] = useState<T>(initialValue);
  const [error, setError] = useState<string>("");

  const onChange = (e: React.ChangeEvent<HTMLInputElement>) => {
    const newValue =
      type === "checkbox"
        ? (e.target.checked as unknown as T)
        : (e.target.value as T);
    setValue(newValue);
    setError("");
  };

  // 입력 값의 유효성을 검사하는 함수, 결과에 따라 true 또는 false를 반환
  const validate = (): boolean => {
    const validationError = validateFn(value);
    setError(validationError || "");
    return !validationError;
  };

  const reset = () => {
    setValue(initialValue);
    setError("");
  };

  return {
    value, // 현재 입력 값
    error, // 유효성 검사 오류 메시지
    onChange, // 입력 이벤트 핸들러
    validate, // 유효성 검사 함수
    reset, // 상태 초기화 함수
  };
}
```

<br>

- `src/components/Dynamic.tsx`
    - 모든 입력 필드는 `useInputEx` 훅으로 관리

```ts
import useInputEx from "../hooks/useInputEx";

export default function Dynamic() {

  // useInputEx 훅을 사용해 텍스트 필드와 유효성 검사 로직 분리
  const {
    // 이름 입력 필드: 공백을 허용하지 않음
    value: name,
    error: nameError,
    onChange: handleNameChange,
    validate: validateName,
  } = useInputEx<string>({
    initialValue: "",
    validateFn: (value) => {
      if (!value) return "이름은 필수입니다.";
      return undefined;
    },
  });

  const {
    // 이메일 입력 필드: @ 포함 여부 확인
    value: email,
    error: emailError,
    onChange: handleEmailChange,
    validate: validateEmail,
  } = useInputEx<string>({
    initialValue: "",
    validateFn: (value) => {
      if (!value.includes("@")) return "올바른 이메일을 입력하세요.";
      return undefined;
    },
  });

  const {
    // 전화번호 입력 필드: 11자리 숫자만 허용
    value: phone,
    error: phoneError,
    onChange: handlePhoneChange,
    validate: validatePhone,
  } = useInputEx<string>({
    initialValue: "",
    validateFn: (value) => {
      if (!value.match(/^\d{11}$/)) return "전화번호는 11자리여야 합니다.";
      return undefined;
    },
  });

  const {
    // 체크박스 필드: 약관 동의 여부
    value: isAgreed,
    error: isAgreedError,
    onChange: handleAgreeChange,
    validate: validateAgree,
  } = useInputEx<boolean>({
    initialValue: false,
    validateFn: (value) => {
      if (!value) return "약관에 동의해야 합니다.";
      return undefined;
    },
    type: "checkbox",
  });

  const {
    // 라디오 버튼 필드: 성별 선택
    value: gender,
    error: genderError,
    onChange: handleGenderChange,
    validate: validateGender,
  } = useInputEx<string>({
    initialValue: "male",
    validateFn: (value) => {
      if (!value) return "성별을 선택하세요.";
      return "";
    },
    type: "radio",
  });

  const handleSubmit = (e: React.FormEvent) => {
    // 폼 제출 시 전체 필드 유효성 검사
    e.preventDefault();
    if (
      validateName() &&
      validateEmail() &&
      validatePhone() &&
      validateAgree() &&
      validateGender()
    ) {
      console.log("폼 제출:", { name, email, phone, isAgreed, gender });
    }
  };

  return (
    <form onSubmit={handleSubmit}>
      <div>
        <label>이름: </label>
        <input type="text" value={name} onChange={handleNameChange} />
        {nameError && <p>{nameError}</p>}
      </div>
      <div>
        <label>이메일: </label>
        <input type="email" value={email} onChange={handleEmailChange} />
        {emailError && <p>{emailError}</p>}
      </div>
      <div>
        <label>전화번호: </label>
        <input type="text" value={phone} onChange={handlePhoneChange} />
        {phoneError && <p>{phoneError}</p>}
      </div>
      <div>
        <label>
          <input
            type="checkbox"
            checked={isAgreed}
            onChange={handleAgreeChange}
          />
          동의합니다
        </label>
        {isAgreedError && <p>{isAgreedError}</p>}
      </div>
      <div>
        <label>성별:</label>
        <label>
          <input
            type="radio"
            name="gender"
            value="male"
            checked={gender === "male"}
            onChange={handleGenderChange}
          />
          남성
        </label>
        <label>
          <input
            type="radio"
            name="gender"
            value="female"
            checked={gender === "female"}
            onChange={handleGenderChange}
          />
          여성
        </label>
        {genderError && <p>{genderError}</p>}
      </div>
      <button type="submit">제출</button>
    </form>
  );
}
```

<br>

<hr>

<br>

## 04. 폼 검증
- 사용자가 입력한 값이 유효한지 확인하고, 올바르지 않은 경우 경고를 표시하거나 폼 제출을 막는 작업

<br>

### 기본 검증
- HTML5에서 제공하는 주요 검증 속성

| **속성** | **설명** | **예** |
| - | - | - |
| `required` | 필수 입력 필드 지정 | `<input type='text' required />`|
| `minlength` | 필수 입력 필드 지정 | `<input type='text' minlength='3' />`|
| `maxlength` | 필수 입력 필드 지정 | `<input type='text' maxlength='3' />`|
| `min` | 필수 입력 필드 지정 | `<input type='number' min='0' />`|
| `max` | 필수 입력 필드 지정 | `<input type='number' max='100' />`|
| `step` | 필수 입력 필드 지정 | `<input type='number' step='2' />`|
| `type` | 필수 입력 필드 지정 | `<input type='email' />`|
| `pattern` | 필수 입력 필드 지정 | `<input type='text' patter='[A-za-z]{3}' />`|

<br>

- `src/components/ValidationForm.tsx`
  - `form.checkValidity()` : 현재 폼 안의 모든 `<input>` 요소가 HTML 속성의 조건을 만족하는지 검사
  - 조건을 모두 만족하면 `true`, 하나라도 만족하지 못하면 `false`

<br>

```ts
import { useState } from "react";

export default function ValidationForm() {
  const [formData, setFormData] = useState({
    username: "",
    email: "",
    age: "",
    birthdate: "",
    phone: "",
    website: "",
    color: "#000000",
    rating: "5",
  });

  const [submitStatus, setSubmitStatus] = useState("");

  const handleSubmit = (e: React.FormEvent) => {
    e.preventDefault();
    const form = e.target as HTMLFormElement;
    // 모든 입력 요소의 유효성 검사 결과 확인
    if (form.checkValidity()) {
      setSubmitStatus("success");
      console.log("폼 데이터:", formData);
    } else {
      setSubmitStatus("error");
    }
  };
  const handleChange = (e: React.ChangeEvent<HTMLInputElement>) => {
    const { name, value } = e.target;
    setFormData((prev) => ({
      ...prev,
      [name]: value,
    }));
  };

  return (
    <div>
      <h1>회원 가입</h1>
      <form onSubmit={handleSubmit}>
        <div>
          <label>
            사용자 이름
            <input
              type="text"
              name="username"
              value={formData.username}
              onChange={handleChange}
              required
              minLength={3}
              maxLength={20}
              pattern="[A-Za-z0-9]+"
              title="3-20자 사이의 영문자와 숫자만 사용할 수 있습니다."
            />
          </label>
        </div>

        ...

    </div>

  )
}
```

<br>

### 커스텀 검증 로직
- 복잡한 검증 조건이나 사용자 정의 메시지가 필요한 경우, 커스텀 혹은 외부 폼 라이브러리를 활용하는 것이 바람직

<br>

- `src/components/validationFormCustom.tsx`

```ts
import { useState } from "react";

export default function ValidationFormCustom() {
  const [formData, setFormData] = useState({
    
    ...

  });

  const [formErrors, setFormErrors] = useState({
    username: "",
    email: "",
    age: "",
    birthdate: "",
    phone: "",
    website: "",
    color: "#000000",
    rating: "5",
  });

  const [submitStatus, setSubmitStatus] = useState("");

  // 커스텀 검증 함수
  const validateUsername = (username: string) => {
    if (!username) return "사용자 이름은 필수입니다.";
    if (username.length < 3 || username.length > 20)
      return "사용자 이름은 3~20자 사이여야 합니다.";
    if (!/^[A-Za-z0-9]+$/.test(username))
      return "사용자 이름은 영문자와 숫자만 포함해야 합니다.";
    return "";
  };

  ...

  const validateForm = () => {
    // 각 필드에 대해 커스텀 검증 함수 실행: 오류 메시지 저장
    const errors = {
      username: validateUsername(formData.username),
      email: validateEmail(formData.email),
      age: validateAge(formData.age),
      birthdate: validateBirthdate(formData.birthdate),
      phone: validatePhone(formData.phone),
      website: validateWebsite(formData.website),
      color: validateColor(formData.color),
      rating: validateRating(formData.rating),
    };

    // 화면에 표시하기 위해 오류 메시지를 상태에 반영
    setFormErrors(errors);

    // 하나라도 오류 메시지가 있으면 isValid를 false로 설정
    return !Object.values(errors).some((msg) => msg !== "");
  };

  const handleSubmit = (e: React.FormEvent) => {
    e.preventDefault();
    if (validateForm()) {
      setSubmitStatus("success");
      console.log("폼 데이터:", formData);
    } else {
      setSubmitStatus("error");
    }
  };

  const handleChange = (e: React.ChangeEvent<HTMLInputElement>) => {
    const { name, value } = e.target;
    setFormData((prev) => ({
      ...prev,
      [name]: value,
    }));
  };

  return (
    <div>
      <h1>회원 가입</h1>
      <form onSubmit={handleSubmit}>
        <div>
          <label>
              사용자 이름
              <input
              type="text"
              name="username"
              value={formData.username}
              onChange={handleChange}
              />
          </label>
          {formErrors.username && <div>{formErrors.username}</div>}
        </div>

        ...

        {submitStatus === "success" && (
            <div>
            <span>폼을 성공적으로 제출했습니다!</span>
            </div>
        )}
        {submitStatus === "error" && (
            <div>
            <span>입력값을 확인하세요.</span>
            </div>
        )}
        <button type="submit">가입하기</button>
      </form>
  </div>
  )
}
```

<br>

### 라이브러리 사용 
- React에서 사용하는 대표적인 폼 검증 라이브러리로는 `Formik`, `React Hook Form`


```bash
$ npm install formik --save
```

<br>

#### `formik` 컴포넌트
- `Field` : `<input>`, `<textarea>` 등 입력 요소를 자동으로 상태와 연결해주는 컴포넌트
- `Form` : HTML의 `<form>` 태그를 대체하는 컴포넌트
- `Formik` : 폼 상태와 검증 로직을 관리하는 컨테이너 역할의 컴포넌트

<br>

```ts
import { Field, Form, Formik } from 'formik';
```

<br>

#### 기본 사용법
- `initialvalues` : 각 입력 요소에 대응하는 초깃값 객체
  - 일반적인 제어 컴포넌트의 `useState`훅으로 상태를 초기화 하는 것과 유사
- `onSubmit` : 폼이 제출될 때 실행할 콜백 함수

```ts
<Formik
    initialValues={{ email: '', password: '', }}
    onSubmit={(values) => { console.log(values); }}
>
</Formik>
```

<br>

#### `<Form>` 컴포넌트
- `Formik`이 내부적으로관리하는 상태와 자동으로 연결 $\rightarrow$ 별도의 `useState` 훅 없이도 입력 값을 쉽게 관리
- `<form>` 요소 처럼 직접 `onSubmit`이벤트 핸들러를 지정하는 대신, `Formik` 컴포넌트에 지정한 `onSubmit` 속성에 콜백함수를 작성 $\rightarrow$ 제출 이벤트가 자동으로 처리
- Enter키 등 기본적인 키보드 동작 지원

<br>

- `name` : 입력 값의 이름
  - `initialValues` 객체의 키와 일치해야 해당 값과 연결
- `type` : 'text', 'email' 등 `<input>` 요소에서 사용하는 타입과 동일
- `component` : 어떤 HTMl 요소를 렌더링할지 지정하는 속성

```ts
<Field name='email' type='email' component='input' />
```

<br>

#### 오류 핸들링
- `name` : 오류 메시지를 표시할 대상 필드의 이름
- `component` : 오류 메시지를 감쌀 HTML 태그 이름
  - `component='div'`로 설정 시, `<div>`안에 오류 메시지 표시

```ts
<ErrorMessage name='키' component='태그' />
```

- **`ErrorMessage` 컴포넌트가 제대로 작동하기 위해서 `name` 속성 값이 다음 세 가지와 모두 일치해야 함**
  - `Formik` 컴포넌트의 `initialvalues` 객체의 키
  - `Field` 컴포넌트의 `name` 속성 값
  - `validate()` 함수에서 반환하는 `errors` 객체의 키

<br>

- **`validate()`는 `onSubmit()` 보다 먼저 실행**
  
  $\rightarrow$ **폼에 오류가 있다면 `onSubmit()`은 실행되지 않음**

<br>

- `src/components/ValidationFormEx2.tsx`

```ts
import { ErrorMessage, Field, Form, Formik } from "formik";

interface ErrorValues {
  email?: string;
  password?: string;
}

export default function ValidationFormEx2() {
  return (
    <>
      <Formik
        initialValues={{
          email: "",
          password: "",
        }}

        onSubmit={(values) => {
          console.log(values);
        }}


        // 폼의 현재값에 오류가 있다면, errors 객체에 오류 메시지를 담아 반환
        // 오류가 없다면 빈 객체를 반환해 폼 제출을 허용
        validate={(values) => {
          const errors: ErrorValues = {};
          if (!values.email) {
            errors.email = "필수 입력 항목입니다.";
          } else if (!values.email.includes("@")) {
            errors.email = "올바르지 않은 이메일 형식입니다.";
          }
          if (!values.password) {
            errors.password = "필수 입력 항목입니다.";
          } else if (values.password.length < 4) {
            errors.password =
              "비밀번호는 대소문자, 특수문자를 포함해 4자 이상이어야 합니다.";
          }
          return Object.keys(errors).length > 0 ? errors : {};
        }}
      >
        {({ isSubmitting }) => (
          <Form>
            <Field
              name="email"
              type="email"
              placeholder="이메일을 입력하세요"
              component="input"
            />
            <ErrorMessage name="email" component="div" />
            <Field
              name="password"
              type="password"
              placeholder="비밀번호를 입력하세요."
              component="input"
            />
            <ErrorMessage name="password" component="div" />
            <button type="submit" disabled={isSubmitting}>
              {isSubmitting ? "로그인중..." : "로그인"}
            </button>
          </Form>
        )}
      </Formik>
    </>
  );
}
```

<br>

- **`ErrorMessage` 컴포넌트는 한 번 오류 메시지가 표시되면, 입력 값이 바뀔 떄마다 실시간으로 메시지를 업데이트**
  
  $\rightarrow$ **잦은 포커스 이동으로 사용자의 불편함 증가 가능**

  $\rightarrow$ **입력 도중에는 메시지를 숨기고, 입력 필드에서 포커스가 빠져나갈 때만 오류를 표시하도록 설정**

<br>

```ts
export default function ValidationFormEx2() {
  return (
    <>
      <Formik
        initialValues={{
          email: "",
          password: "",
        }}

        onSubmit={(values) => {
          console.log(values);
        }}

        validateOnChange={false} // 입력 중에는 유효성 검사 비활성화
        validateOnBlur={true} // 포커스가 빠져나갈 때 유효성 검사 활성화

        ...

      >
      </Formik>
    </>
  )
};
```

<br>

#### 제출 이벤트 처리
- `Formik` 컴포넌트의 `validate` 속성에는 입력 값을 검사하는 유효성 검사 함수를 작성
  
  $\rightarrow$ 이 함수에서 빈 객체를 반환하면 모든 입력 값이 유효하다고 간주하여 `onSubmit` 속성에 등록한 콜백 함수가 실행

- **`onSubmit`**
    - `values` : 사용자가 입력한 값이 담긴 객체. 각 필드의 이름(`name` 속성)과 그에 대응하는 입력 값이 포함
    - `formikHelpers` : `Formik`이 제공하는 다양한 도우미 함수가 들어 있는 객체 (예 : `resetForm()`, `setSubmitting()`)
      - `setSubmitting()` : 현재 폼이 제출 중인 상태인지 아닌지를 설정
        - `setSubmitting(true)`를 호출하면 폼이 제출 중인 상태로 전환되고, 서버 요청이 끝난 뒤에는 `setSubmitting(false)` 를 호출해 제출 완료 상태로 변경
```ts
<Formik
    initialValues={{ email: '', password: '', }}
    onSubmit={(values, formikHelpers) => { console.log(values); }}
>
</Formik>
```

<br>

- **서버 응답을 1초간 지연**

```ts
<Formik
  initialValues = {{ email: '', password: '', }}
  onSubmit={(values, { setSubmitting }) => {
    setSubmitting(true); // 제출 중 상태로 전환
    setTimeout(() => { // 지연 처리
      console.log('폼 데이터', values);
      setSubmitting(false); // 제출 완료
    }, 1000);
  }}
>
</Formik>
```

<br>

- Form 컴포넌트를 일반 JSX처럼 사용하는 방식 외에도 콜백 함수 형태로 렌더링

```ts
export default function ValidationFormEx2() {
  return(
    <>
      <Formik
        initialValues={{ email: '', password: '', }}
        onSubmit={(values, { setSubmitting }) => {

          ...

        }}
      >
        { ({ isSubmitting }) => (
          <Form>
            
            ...

            <button type='submit' disabled={isSubmitting}>{isSubmitting ? '로그인 중...' : '로그인'}</button>
          </Form>
        )}
      </Formik>
    </>
  )
}
```

<br>

<hr>

<br>

## 05. React 19에서 `ref`
- `ref` 객체를 사용하면 폼 요소의 값을 직접 제억하거나 DOM 요소에 접근해 조작할 수 있음

<br>

### `ref` 객체의 컴포넌트 전달 방식
- **React 18까지는 `ref`를 일반 `props` 처럼 하위 컴포넌트에 직접 전달하는 것이 허용되지 않았으나,**
  
  **React 19부터는 `ref`를 일반 `props` 처럼 컴포넌트에 전달 가능**

<br>

#### React 18 이전
- `ref` 객체는 일반적인 `props` 처럼 컴포넌트에 바로 전달할 수 없는 속성이기 때문에, React 18까지는 `ref`를 하위 컴포넌트에 전달하려면 반드시 `forwardRef()` 함수를 사용해 별도로 처리
-**`forwardRef()` : 부모 컴포넌트에서 전달한 `ref` 객체를 자식 컴포넌트 내부의 DOM 요소에 연결할 수 있게 하는 함수**

<br>

#### React 19 이후
- React 19부터는 ref 객체도 일반 `prop` 처럼 컴포넌트에 전달할 수 있고, 컴포넌트 내부에서 간단하게 구조 분해 할당으로 꺼내 사용할 수 있음

<br>

```ts
type InputProps = React.ComponentPropsWithRef<"input"> & {
  label: string;
};
export default function Input({ label, ref, ...rest }: InputProps) {
  const id = rest.id || rest.name; // id를 label과 연결하기 위해 설정
  return (
    <div className="input-group">
      {/* <div>로 감싸 입력 그룹을 시각적으로 묶어줌 */}
      {label && <label htmlFor={id}>{label}</label>}
      <input ref={ref} {...rest} />
    </div>
  );
}
```

<br>

- 부모 컴포넌트 

```ts
import { useRef, useState } from "react";
import Input from "./components/InputReact19";
import Button from "./components/Button";

export default function App() {
  const userInputEl = useRef<HTMLInputElement>(null);
  const passwordInputEl = useRef<HTMLInputElement>(null);

  // 아이디, 비밀번호, 오류, 로딩 상태 관리
  const [username, setUsername] = useState("");
  const [password, setPassword] = useState("");
  // 로그인 폼 제출 이벤트 핸들러
  const handleLogin = async (e: React.FormEvent<HTMLFormElement>) => {
    e.preventDefault(); // 기본 폼 제출 동작 방지
    if (!username) {
      // username 필드가 비어 있으면 오류 메시지 출력
      alert("아이디를 입력하세요.");
      userInputEl.current?.focus();
      return;
    }
    if (!password) {
      // password 필드가 비어 있으면 오류 메시지 출력
      alert("비밀번호를 입력하세요.");
      passwordInputEl.current?.focus();
      return;
    }
  };
  return (
    <div>
      <h2>로그인</h2>
      <form onSubmit={handleLogin}>
        <Input
          ref={userInputEl}
          label="아이디"
          type="text"
          id="username"
          value={username}
          onChange={(e) => setUsername(e.target.value)}
          placeholder="아이디 입력"
        />
        <Input
          ref={passwordInputEl}
          label="비밀번호"
          type="password"
          id="password"
          value={password}
          onChange={(e) => setPassword(e.target.value)}
          placeholder="비밀번호 입력"
        />
        <Button type="submit">로그인</Button>
      </form>
    </div>
  );
}
```

<br>

<hr>